![](2024-10-01-21-16-42.png)

If something invoke api, else don't do anything.

Agent 1 - Database Agent with access to the database and ability to retrieve multiple data points - 2 data points

Agent 2 - Comparison: user input compared to database value

In [2]:
import pandas as pd

df = pd.read_csv("./db-example.csv")

In [3]:
df

,code,description,expiration_date,is_active
0,SAVE10,10% off purchase,2024-12-31,True
1,FREESHIP,Free shipping,2024-06-30,False
2,WELCOME15,15% off first order,2025-01-01,True
3,BLACKFRIDAY20,20% off Black Friday sale,2024-11-29,True
4,SUMMER5,$5 off summer sale,2024-08-31,False
5,NEWYEAR30,30% off New Year's sale,2025-01-02,True
6,CYBERDEAL,25% off Cyber Monday,2024-12-02,True
7,SPRINGFREE,Free gift with purchase,2024-04-30,True
8,VIP50,50% off for VIP members,2024-12-31,False
9,HOLIDAY15,15% off holiday sale,2024-12-25,True


In [5]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

In [6]:
config_list = config_list_from_json("OAI_CONFIG_LIST")

In [23]:
import os
from autogen import ConversableAgent, register_function

# Create assistant and user proxy agents
assistant = ConversableAgent(
    name="DB expert",
    system_message="You are a database expert specialized in retrieving data from a database in ./db-example.csv and then if the user submits a valid code you make an api call and if not you return an error.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
)

import csv


def get_discount_code_details(code_value: str, csv_filename: str = 'db-example.csv'):
    """
    Retrieves the details of a discount code from a CSV file.

    Parameters:
        code_value (str): The discount code to search for.
        csv_filename (str): The path to the CSV file containing discount codes.

    Returns:
        dict or None: A dictionary with the discount code details if found, otherwise None.
    """
    try:
        with open(csv_filename, mode='r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row['code'] == code_value:
                    return row
        return None
    except FileNotFoundError:
        print(f"The file {csv_filename} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


get_discount_code_details("SAVE10")

{'code': 'SAVE10',
 'description': '10% off purchase',
 'expiration_date': '2024-12-31',
 'is_active': 'true'}

In [26]:
# Register the calculator function with both agents
register_function(
    get_discount_code_details,
    caller=assistant,
    executor=user_proxy,
    name="retrieve-code-from-db",
    description="A tool to retrieve code information from a database",
)

The return type of the function 'get_discount_code_details' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
/Users/greatmaster/miniconda3/envs/oreilly-autogen/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:2573: UserWarning: Function 'retrieve-code-from-db' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/greatmaster/miniconda3/envs/oreilly-autogen/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:2492: UserWarning: Function 'retrieve-code-from-db' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [27]:
def make_fake_api_call(api_call_value="some api call json thing"):
    print("Making api call value!")
    return api_call_value

register_function(
    make_fake_api_call,
    caller=assistant,
    executor=user_proxy,
    name="make-api-call",
    description="A tool to make an api call",
)

The return type of the function 'make_fake_api_call' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The following parameters of the function 'make_fake_api_call' with default values are not annotated: 'api_call_value'.
/Users/greatmaster/miniconda3/envs/oreilly-autogen/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:2573: UserWarning: Function 'make-api-call' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/greatmaster/miniconda3/envs/oreilly-autogen/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:2492: UserWarning: Function 'make-api-call' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [28]:
user_proxy.initiate_chat(assistant, message="Hi I have my promo code which is: SAVE10")

User (to DB expert):

Hi I have my promo code which is: SAVE10

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
DB expert (to User):

***** Suggested tool call (call_fMX06OfNfQpruoBm0i6zNQu4): retrieve-code-from-db *****
Arguments: 
{"code_value":"SAVE10"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION retrieve-code-from-db...
User (to DB expert):

User (to DB expert):

***** Response from calling tool (call_fMX06OfNfQpruoBm0i6zNQu4) *****
{"code": "SAVE10", "description": "10% off purchase", "expiration_date": "2024-12-31", "is_active": "true"}
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
DB expert (to User):

Your promo code "SAVE10" is v

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[6].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[6].name', 'code': 'invalid_value'}}

In [29]:
user_proxy.initiate_chat(assistant, message="Hi I have my promo code which is: FREESHIP")

User (to DB expert):

Hi I have my promo code which is: FREESHIP

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
DB expert (to User):

***** Suggested tool call (call_oOPlDacjn4DRa6NpA6nSxTGV): retrieve-code-from-db *****
Arguments: 
{"code_value":"FREESHIP"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION retrieve-code-from-db...
User (to DB expert):

User (to DB expert):

***** Response from calling tool (call_oOPlDacjn4DRa6NpA6nSxTGV) *****
{"code": "FREESHIP", "description": "Free shipping", "expiration_date": "2024-06-30", "is_active": "false"}
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
DB expert (to User):

The promo code "FREESHIP"

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[4].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[4].name', 'code': 'invalid_value'}}

In [17]:
df

,code,description,expiration_date,is_active
0,SAVE10,10% off purchase,2024-12-31,True
1,FREESHIP,Free shipping,2024-06-30,False
2,WELCOME15,15% off first order,2025-01-01,True
3,BLACKFRIDAY20,20% off Black Friday sale,2024-11-29,True
4,SUMMER5,$5 off summer sale,2024-08-31,False
5,NEWYEAR30,30% off New Year's sale,2025-01-02,True
6,CYBERDEAL,25% off Cyber Monday,2024-12-02,True
7,SPRINGFREE,Free gift with purchase,2024-04-30,True
8,VIP50,50% off for VIP members,2024-12-31,False
9,HOLIDAY15,15% off holiday sale,2024-12-25,True
